In [322]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer  # Add this line
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer, KNNImputer, IterativeImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [323]:
dir = 'prepared_data.csv'
df = pd.read_csv(dir)
df

,alder,utdanning,inntekt,kjønn,etnisitet,sykehusdød,oppholdslengde,blodtrykk,hvite_blodlegemer,hjertefrekvens,...,kreft_yes,sykdom_CHF,sykdom_COPD,sykdom_Cirrhosis,sykdom_Colon_Cancer,sykdom_Coma,sykdom_Lung_Cancer,sykdom_MOSF w/Malig,lungfunction_mapped,blod_ph_kategorisert
0,60.33899,12.0,1.0,0,0,1,4,43.0,17.097656,112.0,...,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,-1.0
1,52.74698,12.0,0.0,0,0,0,17,70.0,8.500000,88.0,...,False,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2,42.38498,11.0,0.0,0,0,0,3,75.0,9.099609,88.0,...,True,0.0,0.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN
3,93.01599,14.0,NaN,1,0,1,4,110.0,10.398438,101.0,...,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
4,62.37097,14.0,2.0,1,0,0,9,78.0,11.699219,120.0,...,False,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7729,66.07300,8.0,NaN,1,0,0,23,109.0,7.399414,104.0,...,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
7730,55.15399,11.0,NaN,0,0,0,29,43.0,NaN,0.0,...,False,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,-1.0
7731,70.38196,NaN,NaN,1,0,0,8,111.0,8.398438,83.0,...,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
7732,47.01999,13.0,NaN,1,0,1,7,99.0,7.599609,110.0,...,True,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [324]:
# Definer X og y
X = df.drop('oppholdslengde', axis=1)
y = df['oppholdslengde']

# 1. Splitt først dataene i 70% treningssett og 30% for resten (validering + test)
X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Splitt deretter de resterende 30% i to like deler: 15% valideringssett og 15% testsett
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42)

train_df = pd.concat([X_train, y_train], axis=1)

train_df

,alder,utdanning,inntekt,kjønn,etnisitet,sykehusdød,blodtrykk,hvite_blodlegemer,hjertefrekvens,respirasjonsfrekvens,...,sykdom_CHF,sykdom_COPD,sykdom_Cirrhosis,sykdom_Colon_Cancer,sykdom_Coma,sykdom_Lung_Cancer,sykdom_MOSF w/Malig,lungfunction_mapped,blod_ph_kategorisert,oppholdslengde
6964,74.12500,12.0,0.0,1,0,0,77.0,8.898438,60.0,24.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,5
5117,65.26495,12.0,1.0,1,0,1,46.0,18.500000,160.0,38.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.0,5
7258,71.98395,12.0,NaN,1,0,0,115.0,10.500000,74.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,45
3680,52.26599,20.0,3.0,1,2,0,77.0,10.398438,60.0,20.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,19
7571,69.77100,12.0,3.0,1,0,0,57.0,16.898438,165.0,40.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,58.71597,9.0,0.0,1,0,0,117.0,11.398438,76.0,24.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7
5390,81.90594,NaN,0.0,0,0,0,103.0,10.199219,64.0,16.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,33
860,60.37000,12.0,3.0,0,0,0,73.0,10.000000,80.0,20.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,6
7603,80.05499,8.0,1.0,1,0,0,127.0,8.099609,102.0,38.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,4


In [325]:
train_df.isnull().sum()

alder                               5
utdanning                         978
inntekt                          1793
kjønn                               0
etnisitet                           0
sykehusdød                          0
blodtrykk                           0
hvite_blodlegemer                 132
hjertefrekvens                      0
respirasjonsfrekvens                0
kroppstemperatur                    0
serumalbumin                     1992
bilirubin                        1540
kreatinin                          42
natrium                             0
glukose                          2660
blodurea_nitrogen                2568
urinmengde                       2864
dødsfall                            0
antall_komorbiditeter               0
koma_score                          0
fysiologisk_score                   0
apache_fysiologisk_score            0
overlevelsesestimat_2mnd            0
overlevelsesestimat_6mnd            0
diabetes                            0
demens      

In [326]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline


model = make_pipeline(SimpleImputer(strategy='mean'), 
                      KNeighborsClassifier(n_neighbors=50))

model.fit(X_train, y_train)
prediction = model.predict(X_val)
accuracy_score(y_val, prediction)
print('Accuracy score for mean:', accuracy_score(y_val, prediction))

model = make_pipeline(SimpleImputer(strategy='median'),
                        KNeighborsClassifier(n_neighbors=50))
model.fit(X_train, y_train)
prediction = model.predict(X_val)
accuracy_score(y_val, prediction)
print('Accuracy score for median:', accuracy_score(y_val, prediction))


Accuracy score for mean: 0.07844827586206897
Accuracy score for median: 0.07844827586206897


In [327]:
model = make_pipeline(KNNImputer(n_neighbors=50), 
                      KNeighborsClassifier(n_neighbors=50))

model.fit(X_train, y_train)
prediction = model.predict(X_val)
accuracy_score(y_val, prediction)

0.09224137931034483

In [328]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, mean_squared_error


# Boolean variabel som styrer om GridSearchCV skal kjøres eller ikke
run_grid_search = False  # Sett til False hvis du ikke vil kjøre GridSearchCV

if run_grid_search:
    # Definer pipeline med standardisering, imputering og KNN-klassifisering
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Standardiserer dataene før imputering og klassifisering
        ('imputer', SimpleImputer()),  # Imputeren vil justeres i param_grid
        ('knn', KNeighborsClassifier())  # KNN-algoritmen, n_neighbors settes i param_grid
    ])

    # Utvid param_grid for å inkludere flere alternativer for KNNImputer
    param_grid = {
        'imputer': [
            SimpleImputer(strategy='mean'),
            SimpleImputer(strategy='median'),
            KNNImputer(n_neighbors=10),  # Kan også bruke grid search for n_neighbors
            KNNImputer(n_neighbors=20),
            KNNImputer(n_neighbors=50)
        ],
        'knn__n_neighbors': [3, 5, 10, 20, 30, 50, 200]  # Prøv flere verdier for KNN
    }

    # Bruk stratified 5-fold cross-validation for ubalanserte klasser
    cv = StratifiedKFold(n_splits=5)

    # Sett opp GridSearchCV med kryssvalidering og flere scorer
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=cv, scoring='accuracy', n_jobs=-1, verbose=1  # Verbose for å logge fremdrift
    )

    # Tren modellen med GridSearchCV på treningsdataen
    grid_search.fit(X_train, y_train)

    # Print de beste parameterne og score
    print(f"Beste parametere: {grid_search.best_params_}")
    print(f"Beste score (accuracy): {grid_search.best_score_}")

    # Evaluer den beste modellen på valideringssettet
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_val)

    # Beregn accuracy på valideringsdataen
    accuracy = accuracy_score(y_val, y_pred)
    print(f"Accuracy på valideringssettet: {accuracy}")
else:
    print("GridSearchCV kjører ikke. Juster 'run_grid_search' for å kjøre den.")




GridSearchCV kjører ikke. Juster 'run_grid_search' for å kjøre den.


In [329]:
# Fyller inn manglende verdier med KNNImputer

from sklearn.impute import KNNImputer

# Separér funksjonsvariablene (X_train) fra målet (y_train)
X_train_impute = train_df.drop('oppholdslengde', axis=1)

# Sett opp KNNImputer
imputer = KNNImputer(n_neighbors=50)

# Imputer manglende verdier i X_train
X_train= imputer.fit_transform(X_train_impute)

# Konverter resultatet tilbake til en DataFrame for bedre lesbarhet
X_train = pd.DataFrame(X_train, columns=X_train_impute.columns)

# Kombiner X_train_imputed og y_train tilbake til train_df
train_df = pd.concat([X_train, y_train.reset_index(drop=True)], axis=1)

train_df
train_df.isnull().sum()


alder                            0
utdanning                        0
inntekt                          0
kjønn                            0
etnisitet                        0
sykehusdød                       0
blodtrykk                        0
hvite_blodlegemer                0
hjertefrekvens                   0
respirasjonsfrekvens             0
kroppstemperatur                 0
serumalbumin                     0
bilirubin                        0
kreatinin                        0
natrium                          0
glukose                          0
blodurea_nitrogen                0
urinmengde                       0
dødsfall                         0
antall_komorbiditeter            0
koma_score                       0
fysiologisk_score                0
apache_fysiologisk_score         0
overlevelsesestimat_2mnd         0
overlevelsesestimat_6mnd         0
diabetes                         0
demens                           0
lege_overlevelsesestimat_2mnd    0
lege_overlevelsesest

In [330]:
from sklearn.feature_selection import mutual_info_regression
import pandas as pd

X = train_df.drop(columns=['oppholdslengde'])
y = train_df['oppholdslengde']

# Beregn Mutual Information for regresjon
mi = mutual_info_regression(X, y)

# Lag en DataFrame for å vise Mutual Information for hver funksjon
mi_df = pd.DataFrame({
    'Feature': X.columns,
    'Mutual Information': mi
})

# Sorter etter Mutual Information verdi
mi_df_sorted = mi_df.sort_values(by='Mutual Information', ascending=False)

# Skriv ut de sorterte resultatene
print(mi_df_sorted)

#for å få innsikt i hvilke variabler som har størst informasjonsverdi for prediksjon av oppholdslengde.
#Hvis du har kategoriske variabler, må du sikre at de er riktig kodet numerisk før du bruker denne metoden.

                          Feature  Mutual Information
21              fysiologisk_score            0.101788
22       apache_fysiologisk_score            0.095455
30                            adl            0.055258
23       overlevelsesestimat_2mnd            0.053156
38             sykdom_Lung_Cancer            0.044556
20                     koma_score            0.043441
28  lege_overlevelsesestimat_6mnd            0.041497
24       overlevelsesestimat_6mnd            0.037137
9            respirasjonsfrekvens            0.033951
27  lege_overlevelsesestimat_2mnd            0.033750
31               kreft_metastatic            0.032298
11                   serumalbumin            0.024537
12                      bilirubin            0.024046
29                     dnr_status            0.021482
5                      sykehusdød            0.018138
17                     urinmengde            0.016801
25                       diabetes            0.011561
14                        na

#### Trener Grunnlinjemodell

In [331]:
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error


baseline = DummyRegressor(strategy='mean')  # 'mean' predikerer gjennomsnittet
baseline.fit(X_train, y_train)
y_pred_baseline = baseline.predict(X_val)
rmse_baseline = np.sqrt(mean_squared_error(y_val, y_pred_baseline))
rmse_baseline

np.float64(20.603790732142762)

#### RandomForestRegression

In [346]:
run_grid_search = False  # Sett til True for å kjøre GridSearchCV

if run_grid_search:
    param_grid = {
        'n_estimators': [250, 300, 400, 440],
        'max_depth': [None, 5, 10, 20, 50],
        'min_samples_split': [2, 5, 10, 20],
        'min_samples_leaf': [1, 2, 5, 10],
        'oob_score': [True, False],
        'max_features': ['auto', 'sqrt', 'log2']
    }
    model = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=3,               # 3-fold kryssvalidering
        n_jobs=-1,           # Bruk alle tilgjengelige kjerner
        verbose=1            # For å vise fremdriften
    )
    grid_search.fit(X_train, y_train)
    print(f"Beste parametere: {grid_search.best_params_}")
    y_pred = grid_search.best_estimator_.predict(X_val)
    rmse_t = np.sqrt(mean_squared_error(y_val, y_pred))
    print(f"RMSE på valideringssettet: {rmse_t}")
else:
    print("GridSearchCV kjører ikke. Sett 'run_grid_search' til True for å kjøre den.")



Fitting 3 folds for each of 1920 candidates, totalling 5760 fits


/Users/asegrethe/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/asegrethe/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.1 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/asegrethe/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estim

Beste parametere: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 300, 'oob_score': True}
RMSE på valideringssettet: 18.821638635843872


#### Trener modeller 
#### Bruker regression side jeg vil predikere tall


In [348]:
#Logistisk regresjon 

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer


param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] } 
clf = GridSearchCV(LogisticRegression(penalty='l2'), param_grid)
GridSearchCV(cv=None,
             estimator=LogisticRegression(C=1.0, intercept_scaling=1,   
               dual=False, fit_intercept=True, penalty='l2', tol=0.0001),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]})

clf.fit(X_train, y_train)
print(f"Beste parametere: {clf.best_params_}")
y_pred = clf.predict(X_val)
rmse_l=np.sqrt(mean_squared_error(y_val, y_pred))
print(f"RMSE på valideringssettet: {rmse_l}")



/Users/asegrethe/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/model_selection/_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/Users/asegrethe/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/asegrethe/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterat

Beste parametere: {'C': 0.01}


/Users/asegrethe/miniconda3/envs/INF161/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values